In [ ]:

## Libraries Prequisities
!pip install scikit-learn
!pip install numpy==1.16.1

In [ ]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from tensorflow.keras.callbacks import Callback

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
np.random.seed(12227)

In [ ]:
n_ep = 200
loss = 0.2
bs = 1000

In [ ]:
def custom_stopping(value=0.5, verbose=0):
    early = EarlyStoppingByLossVal(monitor='val_loss', value=value, verbose=verbose)
    return early

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_acc', value=0.95, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        # if current is None:
        # warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [ ]:
def DataPreparation(data_input_file):
  print('Kasnesis  et al. 2018 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  Y = data['y']
  folds = data['folds']
 
  return X,Y,folds

In [ ]:
X,Y,f=DataPreparation('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

In [ ]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [ ]:
def RunKasnesis(data_input_file):
  X,Y,folds,= DataPreparation(data_input_file)
  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [ ]:
def perception_net(input_dim, num_classes, filters=(48, 96, 96), dilation=False, separate_modalities=False):
    """ PerceptionNet Model.
    See paper: https://arxiv.org/abs/1811.00170
    Arguments:
        input_dim: 2D input array, where the first value is the number of signals
            and the second the time steps.
        num_classes: Number of classes to predicts.
        filters: Optional, amount of feature maps employed by the convolutions.
        dilation: Optional, true whether dilation should be employed in the
            1D Conv blocks.
    """

    if dilation:
        kernel_width = 8
        dilation_rate = 2
    else:
        kernel_width = 15
        dilation_rate = 1

    input_tensor = tf.keras.Input(shape=(1,input_dim[0], input_dim[1]))
    x = tf.keras.layers.Reshape(target_shape=(1,input_dim[0], input_dim[1]))(input_tensor)

    # First "Conv1D" block
    x = tf.keras.layers.Conv2D(filters=filters[0],
               kernel_size=(1,kernel_width), # height, width
               dilation_rate=dilation_rate,
               kernel_initializer='random_uniform',
               padding="same")(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(1,2),
                  strides=(1,2))(x)
    x = tf.keras.layers.Dropout(rate=0.4)(x)

    # Second "Conv1D" block
    x = tf.keras.layers.Conv2D(filters=filters[1],
               kernel_size=(1,kernel_width), #height, width
               dilation_rate=dilation_rate,
               kernel_initializer='random_uniform',
               padding="same")(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(1,2),
                  strides=(1,2))(x)
    x = tf.keras.layers.Dropout(rate=0.4)(x)

    # Late sensor fusion
    x = tf.keras.layers.Conv2D(filters=filters[2],
               kernel_size=(3,15),
               strides=(3,1),
               kernel_initializer='random_uniform',
               padding="same")(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(rate=0.4)(x)

    # Classification
    x = tf.keras.layers.Dense(units=num_classes, activation='softmax')(x)

    return tf.keras.Model(input_tensor, x, name='PerceptionNet')


In [ ]:
batch_size = 64
epochs = 2000
validation_split = 0.15
stopping_patience = 100
model_save_frequency = 50
# AdaDelta parameters
learning_rate = 1.0
rho = 0.95
epsilon = 1e-08


In [ ]:
def Train(X,y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _, _,img_rows, img_cols = X.shape
  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    model=perception_net(input_dim=(img_rows, img_cols), num_classes=n_class, filters=(12,24,24), dilation=False)
    optimizer = tf.keras.optimizers.Adadelta(lr=learning_rate,rho=rho, epsilon=epsilon)
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=stopping_patience, restore_best_weights=True),
        
    ]

    model.compile(loss="categorical_crossentropy",
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    model.fit(X_train, y[train_idx], batch_size=batch_size,epochs=epochs, validation_split=validation_split, shuffle=True,callbacks=callbacks)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1

In [ ]:
tf.keras.backend.set_image_data_format('channels_first')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/FNOW/MHEALTH.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOTO/MHEALTH.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/SNOW/MHEALTH.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOSO/USCHAD.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOTO/USCHAD.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/FNOW/USCHAD.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/SNOW/USCHAD.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD1_1s.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD1_1s.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD1_1s.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD2_1s.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD2_1s.npz')

In [ ]:
RunKasnesis('/content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD2_1s.npz')